/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytr

In [7]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import requests
from PIL import Image
from io import BytesIO
import os
import csv

from transformers import ViTFeatureExtractor, ViTForImageClassification
import torch

model = ViTForImageClassification.from_pretrained('nateraw/vit-age-classifier')
feature_extractor = ViTFeatureExtractor.from_pretrained('nateraw/vit-age-classifier')

image_directories = {
    "clean": '/Users/samin/Desktop/Classes/9.60/9.60-Project/clean_dataset/images',
    "adversarial": '/Users/samin/Desktop/Classes/9.60/9.60-Project/adversarial_dataset/images'
}

def evaluate_model(image_dir, csv_file_path, dataset_name):
    predictions = []

    for image_name in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image_name)
        try:
            with Image.open(image_path) as im:
                inputs = feature_extractor(images=im, return_tensors="pt")
                outputs = model(**inputs)
                proba = torch.softmax(outputs.logits, dim=1)
                pred_class = proba.argmax(1).item()
                predictions.append((image_name, pred_class))
        except Exception as e:
            print(f"An error occurred with image {image_name}: {e}")

    actual_labels = {}
    with open(csv_file_path, mode='r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        for row in csvreader:
            actual_labels[row['image_name']] = int(row['class_label'])

    correct_predictions = 0
    for image_name, pred_class in predictions:
        if image_name in actual_labels:
            actual_class = actual_labels[image_name]
            if pred_class == actual_class:
                correct_predictions += 1

    accuracy = correct_predictions / len(predictions) if predictions else 0
    print(f"Accuracy for {dataset_name}: {accuracy * 100:.2f}%")

# Evaluate model on both datasets
for dataset_name, image_dir in image_directories.items():
    csv_file_path = f'/Users/samin/Desktop/Classes/9.60/9.60-Project/{dataset_name}_dataset/labels.csv'
    evaluate_model(image_dir, csv_file_path, dataset_name)

Accuracy for clean: 62.00%
Accuracy for adversarial: 58.00%
